<a href="https://colab.research.google.com/github/ivanmladek/Sentinel-Intelligence-Codex/blob/main/process_refactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Processing Pipeline

## 1. Setup and Dependencies

In [ ]:
#@title Install System Dependencies
!apt-get install -y poppler-utils tesseract-ocr libmagic-dev unrar

In [ ]:
#@title Install Python Libraries (Part 1)
!pip install numpy==1.26.4

In [ ]:
#@title Install Python Libraries (Part 2)
!pip install transformers==4.38.2 pyarrow==14.0.1 timm==0.5.4 requests==2.31.0 albumentations==1.0.0 git+https://github.com/facebookresearch/nougat
!pip install textblob langdetect beautifulsoup4 huggingface_hub tqdm pandas

## 2. Imports and Configuration

In [ ]:
import os
import re
import json
import logging
import shutil
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed

import nltk
import pandas as pd
import requests
from bs4 import BeautifulSoup
from huggingface_hub import HfApi
from langdetect import detect, LangDetectException
from nltk.corpus import words, brown
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
from tqdm import tqdm
from google.colab import drive

# --- Configuration ---
BASE_URL = "https://the-eye.eu/public/Books/Bibliotheca%20Alexandrina/"
HUGGING_FACE_REPO = "ivanmladek/Sentinel-Intelligence-Codex"  # Replace with your Hugging Face repo
GARBAGE_THRESHOLD = 0.8
LENWORD = 50

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Download NLTK Data ---
nltk.download('punkt')
nltk.download('words')

## 3. Helper Functions

### 3.1. File and Web Operations

In [ ]:
def get_file_list(url):
    """Get a list of files from a URL."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return [link.get('href') for link in soup.find_all('a') if link.get('href').endswith('.rar')]

def download_file(url, output_path):
    """Download a file from a URL."""
    if os.path.exists(output_path):
        logger.info(f"{output_path} already exists. Skipping download.")
        return
    response = requests.get(url, stream=True)
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

def extract_rar(file_path, output_path):
    """Extract a RAR file."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        subprocess.run(['unrar', 'x', file_path, output_path], check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        logger.error(f"Error extracting {file_path}: {e.stderr}")

def sanitize_filename(filename):
    """Sanitize a filename."""
    return re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)

### 3.2. PDF Processing (Nougat)

In [ ]:
def process_pdf(pdf_path, output_dir):
    """Process a single PDF file with Nougat."""
    sanitized_filename = sanitize_filename(os.path.basename(pdf_path))
    mmd_path = os.path.join(output_dir, f"{os.path.splitext(sanitized_filename)[0]}.mmd")
    if os.path.exists(mmd_path):
        logger.info(f"{mmd_path} already exists. Skipping Nougat processing.")
        return mmd_path

    try:
        subprocess.run(['nougat', pdf_path, '-o', output_dir, '--no-skipping', '--recompute'], check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        logger.error(f"Error processing {pdf_path} with Nougat: {e.stderr}")
        return None
    return mmd_path

### 3.3. Text Cleaning and Quality Control

In [ ]:
def clean_text(text):
    """Clean the extracted text."""
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    text = re.sub(r'\[[^\]]*\]', '', text)
    text = re.sub(r'\(\d+\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[[A-Za-z0-9]+\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\([\w\s]+et\s+al\., \d{4}\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(\w+\s+and\s+\w+\s+\d{4}\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(see\s+equations\s+\(\d+\)\s+and\s+\(\d+\)\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(\w+\s+et\s+al\., \d{4};\s*\w+\s+et\s+al\., \d{4}\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Table\s+\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[FIGURE:[^]]+\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[\d+(,\s*\d+)*\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[.*arxiv.*\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[^\x00-\x7F]+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[\.,;:!?]{2,}', '', text, flags=re.IGNORECASE)
    return text

def is_garbage(text, threshold=GARBAGE_THRESHOLD, lenword=LENWORD):
    """Check if the text is garbage."""
    if not text or len(text.split()) < 10:
        return True
    try:
        if detect(text) != 'en':
            return True
    except LangDetectException:
        return True
    return False

### 3.4. Text Chunking

In [ ]:
def chunk_text(content, max_size=8192):
    """Chunk the text into smaller segments."""
    segments = []
    current_segment = """
    lines = content.split('\n')

    for line in lines:
        if line.startswith(("#", "##", "###")):
            if current_segment:
                segments.extend(split_segment(current_segment.strip(), max_size))
            current_segment = """
        else:
            current_segment += line + " "

    if current_segment:
        segments.extend(split_segment(current_segment.strip(), max_size))

    return segments

def split_segment(segment, max_size):
    """Split a segment into smaller chunks."""
    sentences = sent_tokenize(segment)
    chunks = []
    current_chunk = """

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def process_and_chunk_mmd(mmd_path, output_dir):
    """Process and chunk an MMD file."""
    if not mmd_path or not os.path.exists(mmd_path):
        logger.warning(f"MMD file not found: {mmd_path}. Skipping.")
        return None, None

    cleaned_jsonl_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(mmd_path))[0]}_cleaned.jsonl")
    garbage_jsonl_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(mmd_path))[0]}_garbage.jsonl")

    with open(mmd_path, 'r', encoding='utf-8') as f:
        content = f.read()

    chunks = chunk_text(content)
    with open(cleaned_jsonl_path, 'w') as cleaned_f, open(garbage_jsonl_path, 'w') as garbage_f:
        for chunk in chunks:
            cleaned_chunk = clean_text(chunk)
            if is_garbage(cleaned_chunk):
                garbage_f.write(json.dumps({"text": cleaned_chunk}) + '\n')
            else:
                cleaned_f.write(json.dumps({"text": cleaned_chunk}) + '\n')

    return cleaned_jsonl_path, garbage_jsonl_path

### 3.5. Hugging Face Integration

In [ ]:
def upload_to_huggingface(file_path, repo_id):
    """Upload a file to a Hugging Face repository."""
    api = HfApi()
    api.upload_file(
        path_or_fileobj=file_path,
        path_in_repo=os.path.basename(file_path),
        repo_id=repo_id,
        repo_type="dataset",
    )
    logger.info(f"Uploaded {file_path} to {repo_id}")

## 4. Main Processing Loop

In [ ]:
def main():
    """Main function to process the library."""
    rar_files = get_file_list(BASE_URL)

    with tqdm(total=len(rar_files), desc="Processing RAR Files") as pbar:
        for rar_file in rar_files:
            rar_url = BASE_URL + rar_file
            rar_path = sanitize_filename(rar_file)
            extract_path = os.path.splitext(rar_path)[0]

            download_file(rar_url, rar_path)
            extract_rar(rar_path, extract_path)

            pdf_files = [os.path.join(root, file) for root, _, files in os.walk(extract_path) for file in files if file.endswith('.pdf')]

            for pdf_path in pdf_files:
                mmd_path = process_pdf(pdf_path, extract_path)
                cleaned_jsonl, garbage_jsonl = process_and_chunk_mmd(mmd_path, extract_path)

                if cleaned_jsonl and os.path.exists(cleaned_jsonl):
                    upload_to_huggingface(cleaned_jsonl, HUGGING_FACE_REPO)
                if garbage_jsonl and os.path.exists(garbage_jsonl):
                    upload_to_huggingface(garbage_jsonl, HUGGING_FACE_REPO)

            os.remove(rar_path)
            shutil.rmtree(extract_path)
            pbar.update(1)

if __name__ == "__main__":
    main()